# Comparison between the enrichment scores and experimental evaluation of individual variants

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FormatStrFormatter
import scipy

plt.style.use('/work/projects/project01640/jm/domain_insertion/dipSeq/batch_scripts_JMA/scripts_paper/utils/domain_ins.mplstyle')
plt.rcParams['svg.fonttype'] = 'none'

### Import data

In [ ]:
base = '/work/projects/project01640/jm/domain_insertion/DI_screen'
fig_folder = f'{base}/analysis/figures'
data = pd.read_csv(f'{base}/experimental_rawdata/Wetlab_validation_switches.csv', delimiter=';')
del data['Unnamed: 3']

In [ ]:
data[['protein', 'position', 'insert']] = data['name'].str.split('_', expand=True)
data['state'] = np.where(data['enrichment'] > 0, 1, 0)
data.dropna(inplace=True)
data

In [ ]:
datasets = [['AraC', 'PDZ'], ['AraC', 'LOV'], ['AraC', 'ERD'], ['AraC', 'UNI'], ['AraC', 'YFP'], ['TVMV', 'PDZ'], ['Flp', 'PDZ']]

for combi in datasets:
    try:
        current_data = data[(data['protein'] == combi[0]) & (data['insert'] == combi[1])]
        s, pvalue = scipy.stats.mannwhitneyu(current_data[current_data["state"] == 0]['fluorescence'], current_data[current_data["state"] == 1]['fluorescence'])
        print(pvalue)
        plt.figure(figsize=(2.5,4))
        ax = sns.boxplot(data=current_data, x='state', y='fluorescence', fliersize=0, linewidth=1.5, whiskerprops={'color':'grey'}, 
        capprops={'color':'grey'}, boxprops={'color':'white', 'linewidth':1.5, 'edgecolor':'grey'}, medianprops={'color':'#E60234'})
        ax = sns.stripplot(data=current_data, x='state', y='fluorescence', alpha=1, linewidth=0, color='black')
        sns.despine()
        ax.set_yscale('log')
        plt.title(f'{combi[0]}-{combi[1]}')
        plt.ylabel('RFP/OD600 [a.u.]')
        plt.xticks([0,1], ['Depleted', 'Enriched'])
        if combi[0] == 'TVMV':
            ax.yaxis.set_minor_formatter(FormatStrFormatter("%.1f"))
        plt.xlabel("")
        plt.savefig(f"{fig_folder}/{combi[0]}{combi[1]}_wetlab.svg")

    except:
        pass